# Bytewax

In [1]:
from datetime import timedelta

import bytewax.operators as op
from bytewax.dataflow import Dataflow
from bytewax.connectors.stdio import StdOutSink
from bytewax.testing import TestingSource

ModuleNotFoundError: No module named 'bytewax.operators'

In [18]:
flow = Dataflow("collect")

In [19]:
step1 = op.input("input", flow, TestingSource(list(range(10))))

In [20]:
step2 = op.output('output', step1, StdOutSink())

# Alpaca

In [1]:
import requests
from pprint import pprint
from websocket import create_connection
import json

In [2]:
API_KEY = 'PKF3KMYDUEAUK0S112HX'
SECRET = 'n1hrmo48mJpBoyV7LtVtx6XflUjZymEwd9d9RFev'
ws_url = "wss://stream.data.alpaca.markets/v1beta1/news"

## Batch

In [10]:
import requests

url = "https://data.alpaca.markets/v1beta1/news"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": API_KEY,
    "APCA-API-SECRET-KEY": SECRET
}

params = {'start': '2024-06-27', 'end': '2024-03-29', 'symbols': ['CRM'], 'include_content': True, 'limit': 50}
response = requests.get(url, headers=headers, params=params)

print(response.text)

{"message":"Start should not be before end"}



In [5]:
import requests

url = "https://data.alpaca.markets/v1beta1/news?start=2024-02-01&end=2024-02-10&sort=desc&symbols=CRM&include_content=true&exclude_contentless=true"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": API_KEY,
    "APCA-API-SECRET-KEY": SECRET
}

response = requests.get(url, headers=headers)

print(response.text)

{"news":[{"author":"Benzinga Insights","content":"\u003cp\u003eInvestors with a lot of money to spend have taken a bullish stance on \u003cstrong\u003eSalesforce\u003c/strong\u003e (NYSE:\u003ca class=\"ticker\" href=\"https://www.benzinga.com/stock/CRM#NYSE\"\u003eCRM\u003c/a\u003e).\u003c/p\u003e\n\u003cp\u003eAnd retail traders should know.\u003c/p\u003e\n\u003cp\u003eWe noticed this today when the trades showed up on publicly available options history that we track here at Benzinga.\u003c/p\u003e\n\u003cp\u003eWhether these are institutions or just wealthy individuals, we don't know. But when something this big happens with CRM, it often means somebody knows something is about to happen.\u003c/p\u003e\n\u003cp\u003eSo how do we know what these investors just did? \u003c/p\u003e\n\u003cp\u003eToday, \u003ca href=\"https://pro.benzinga.com/?utm_source=insights\"\u003eBenzinga\u003c/a\u003e's options scanner spotted 42 uncommon options trades for Salesforce.\u003c/p\u003e\n\u003cp\u00

## Stream (but didnt work or keeps running)

In [3]:
conn = create_connection(ws_url)
conn.recv()

'[{"T":"success","msg":"connected"}]'

In [4]:
conn.send(
        json.dumps(
            {
                "action": "auth",
                "key": API_KEY,
                "secret": SECRET,
            }
        )
    )

conn.recv()

'[{"T":"success","msg":"authenticated"}]'

In [5]:
conn.send(
            json.dumps({"action": "subscribe", "news": ['GOOG']})
        )


44

In [7]:
conn.recv()

# Bytewax and alpaca

In [1]:
from datetime import timedelta

import bytewax.operators as op
from bytewax.dataflow import Dataflow
from bytewax.connectors.stdio import StdOutSink
from bytewax.inputs import DynamicSource
from bytewax.outputs import DynamicSink
import requests
import json

In [2]:
API_KEY = 'PKF3KMYDUEAUK0S112HX'
SECRET = 'n1hrmo48mJpBoyV7LtVtx6XflUjZymEwd9d9RFev'

In [3]:
# util
from datetime import datetime, timedelta

def split_date_range(from_date, to_date, num_intervals):
    # Convert input strings to datetime objects
    from_date = datetime.strptime(from_date, '%Y-%m-%d')
    to_date = datetime.strptime(to_date, '%Y-%m-%d')
    
    # Calculate total number of days between from_date and to_date
    total_days = (to_date - from_date).days
    
    # Calculate the number of days per interval
    days_per_interval = total_days / num_intervals
    
    # Initialize list to store result tuples
    result = []
    
    # Generate intervals
    for i in range(num_intervals):
        # Calculate start and end dates for the current interval
        start_date = from_date + timedelta(days=int(i * days_per_interval))
        end_date = from_date + timedelta(days=int((i + 1) * days_per_interval))
        
        # Append tuple (start_date, end_date) to the result list
        result.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    
    return result


In [8]:
class AlpacaBatch(DynamicSource):
    def __init__(self, from_date, to_date, tickers):
        self.from_date = from_date
        self.to_date = to_date
        self.tickers = tickers 
        self.api_key = API_KEY
        self.secret = SECRET
        # to change the string values
        self.url = "https://data.alpaca.markets/v1beta1/news?start=2024-02-01&end=2024-02-10&sort=desc&symbols=CRM&include_content=true&exclude_contentless=true"
    
    def build(self, worker_index, worker_count):
        print('inside build')
        worker_date_range = split_date_range(self.from_date, self.to_date, worker_count)
        current_from_date, current_to_date = worker_date_range[worker_index] # current wormer's value
        return self.build_alpaca_client(current_from_date, current_to_date)

    def build_alpaca_client(self, current_from_date, current_to_date):
         
        headers = {
                        "accept": "application/json",
                        "APCA-API-KEY-ID": self.api_key,
                        "APCA-API-SECRET-KEY": self.secret
                    }
        params = {
            "start": current_from_date,
            "end"  : current_to_date,
            "sort" : "desc",
            "symbols": self.tickers,
            "include_content": True,
            "page_token": 50,
            "limit": 50
        }
    
        response = requests.get(self.url, headers=headers, params= params)

        next_page_token = None
        if response.status_code == 200:  # Check if the request was successful
            # parse response into json
            news_json = response.json()

            # extract next page token (if any)
            next_page_token = news_json.get("next_page_token", None)

        else:
            print(f"Request failed with status code: {response.status_code}")

        self._page_token = next_page_token

        return news_json["news"]


In [9]:
def build():
    flow = Dataflow("alpaca_news")
    alpaca_batch = op.input("input", flow, AlpacaBatch)
    return alpaca_batch

In [10]:
mynews = build()

In [7]:
mynews

Stream(stream_id='alpaca_news.input.down', _scope=_Scope(parent_id='alpaca_news'))

In [2]:
# Transformers

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

d:\Raghu Studies\FinancialAdvisor\.src_financial_advisor\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
import chromadb
from chromadb.config import Settings
client = chromadb.Client(Settings(chroma_db_impl="news",
                                    persist_directory="/content/"
                                ))

ValueError: [91mYou are using a deprecated configuration of Chroma.

[94mIf you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/8g5FESbj for help![0m

In [4]:
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]
embeddings = model.encode(sentences)
print(embeddings.shape)
# => (3, 384)

(3, 384)


In [56]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
db = chromadb.PersistentClient(path=r"D:\Raghu Studies\FinancialAdvisor\chroma_dir")

In [92]:
model_name = "all-MiniLM-L6-v2"
model = embedding_functions.SentenceTransformerEmbeddingFunction(model_name)
collection_name = "finance"
document = 'hello'
# Check if collection exists, if not create it
collection = db.get_or_create_collection(collection_name)

results = collection.query(
    query_texts=["hello"],
    n_results=2
)
import numpy as np
match_list = [d for d in results['distances'][0] if np.abs(d) < 0.0001]
if(not match_list):
    embeddings_sent = model([document])

    collection.add(embeddings=embeddings_sent, documents=[document], ids=['ID11'])



In [119]:
results = collection.get()

In [130]:
results

{'ids': ['ID1', 'ID10', 'ID11', 'ID2', 'ID3'],
 'embeddings': None,
 'metadatas': [None, None, None, None, None],
 'documents': ['I am fine', 'srihari', 'hello', 'Iam fine', 'srihari'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [128]:
pattern = re.compile(r'\d+')

# Extract numbers from each string in results
numbers = "ID"+str(sorted([int(pattern.search(item).group()) for item in results['ids']])[-1] + 1)

In [129]:
numbers

'ID12'

In [107]:
numbers

[1, 2, 3, 10, 11]

In [43]:
match_list = [d for d in results['distances'][0] if np.abs(d) < 0.0001]

In [45]:
match_list

[]

In [53]:
from chromadb.config import Settings

In [54]:
Settings.allow_reset = True

In [131]:
collection.delete(ids=['ID1', 'ID10', 'ID11', 'ID2', 'ID3'])

In [134]:
collection.query(query_texts='Microsoft', n_results=2)

Number of requested results 2 is greater than number of elements in index 0, updating n_results = 0


{'ids': [[]],
 'distances': [[]],
 'metadatas': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

# Without Bytewax

In [17]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Alpaca API credentials
API_KEY = 'PKNA9BZKOZCJQCEFT1NR'
API_SECRET = 'IaJMEQzWTbUqLLpf20JfoomceBMJoQvkGiEKYQ8Q'
BASE_URL = 'https://data.alpaca.markets/v1beta1/news'

# Function to fetch news for a given ticker
def fetch_alpaca_news(ticker, limit=50, start_date=None, end_date=None):
    headers = {
        'APCA-API-KEY-ID': API_KEY,
        'APCA-API-SECRET-KEY': API_SECRET
    }
    
    params = {
        #'symbols': ticker,
        'limit': limit,
        'start': start_date if start_date else (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d'),
        'end': end_date if end_date else datetime.now().strftime('%Y-%m-%d'),
        "include_content": True
    }
    
    
    response = requests.get(BASE_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data['news']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Example usage: Fetch news for a specific ticker (e.g., AAPL)
ticker = 'MSFT'
news_data = fetch_alpaca_news(ticker, limit=10)

# Convert the news data to a pandas DataFrame for easy manipulation
if news_data:
    news_df = pd.DataFrame(news_data)
    
else:
    print("No news data fetched.")


In [22]:
news_df['headline'][2].split("<")

["Trump Vs. Harris: 2024 Election Betting Odds Show 'Love Story' For Vice President After Debate, Taylor Swift Endorsement"]

In [24]:
from unstructured.cleaners.core import (
    clean_extra_whitespace, 
    clean_non_ascii_chars,
    group_broken_paragraphs
)
import re
from unstructured.partition.html import partition_html

In [25]:
def clean_data(contents):
    TAG_RE = re.compile(r'<[^>]+>')
    contents = TAG_RE.sub('', contents)
    contents = group_broken_paragraphs(contents)
    contents = clean_non_ascii_chars(contents)
    contents = clean_extra_whitespace(contents)
    return contents

news_df['clean_data'] = news_df['content'].apply(clean_data)

In [1]:
import chromadb

In [2]:
db = chromadb.PersistentClient(path=r"D:\Raghu Studies\FinancialAdvisor\chroma_dir")

In [3]:
collection_name = "finance"
collection = db.get_or_create_collection(collection_name)

results = collection.query(
    query_texts=["AAPL"],
    n_results=5
)

Add of existing embedding ID: ID1
Add of existing embedding ID: ID2
Add of existing embedding ID: ID10


In [4]:
results

{'ids': [['ID589', 'ID444', 'ID545', 'ID385', 'ID705']],
 'distances': [[0.9993401169776917,
   1.150500774383545,
   1.1746329069137573,
   1.1843395233154297,
   1.2476844148435782]],
 'metadatas': [[None, None, None, None, None]],
 'embeddings': None,
 'documents': [["To gain an edge, this is what you need to know today. Hope Pinned On Apple Intelligence Please click here for an enlarged chart of Apple Inc (NASDAQ:AAPL). Note the following: This article is about the big picture, not an individual stock.&nbsp; The chart of AAPL stock is being used to illustrate the point. The chart shows when the iPhone 15 was launched. The chart shows the move from iPhone 15 launch to today's iPhone 16 launch. Investors are hopeful that the move in AAPL stock from the iPhone 16 launch to the iPhone 17 will be much bigger compared to the stock move between the launch of iPhone 15 and iPhone 16. The chart shows that even though investors have stars in their eyes, for the short term, AAPL stock has tra

In [5]:
results['documents'], len(results['documents'][0])

([["To gain an edge, this is what you need to know today. Hope Pinned On Apple Intelligence Please click here for an enlarged chart of Apple Inc (NASDAQ:AAPL). Note the following: This article is about the big picture, not an individual stock.&nbsp; The chart of AAPL stock is being used to illustrate the point. The chart shows when the iPhone 15 was launched. The chart shows the move from iPhone 15 launch to today's iPhone 16 launch. Investors are hopeful that the move in AAPL stock from the iPhone 16 launch to the iPhone 17 will be much bigger compared to the stock move between the launch of iPhone 15 and iPhone 16. The chart shows that even though investors have stars in their eyes, for the short term, AAPL stock has traced a negative pattern. The chart shows that AAPL stock hit the top of the resistance zone in July. The chart shows that AAPL stock failed to penetrate the resistance zone. The chart shows that after AAPL stock failed to penetrate the resistance zone, it fell all the 

In [2]:
s = """{"key":[["To gain an edge, this is what you need to know today. Hope Pinned On Apple Intelligence Please click here for an enlarged chart of Apple Inc (NASDAQ:AAPL). Note the following: This article is about the big picture, not an individual stock.&nbsp; The chart of AAPL stock is being used to illustrate the point. The chart shows when the iPhone 15 was launched. The chart shows the move from iPhone 15 launch to today's iPhone 16 launch. Investors are hopeful that the move in AAPL stock from the iPhone 16 launch to the iPhone 17 will be much bigger compared to the stock move between the launch of iPhone 15 and iPhone 16. The chart shows that even though investors have stars in their eyes, for the short term, AAPL stock has traced a negative pattern. The chart shows that AAPL stock hit the top of the resistance zone in July. The chart shows that AAPL stock failed to penetrate the resistance zone. The chart shows that after AAPL stock failed to penetrate the resistance zone, it fell all the way to the bottom band of the support zone. The chart shows that AAPL stock rallied after falling to the bottom of the support zone. The chart shows that the rally stopped at the low band of the resistance zone. Investors are mostly focused on hopes that Apple Intelligence, the name for Apple's artificial intelligence, will start a new super cycle of iPhone sales, and thus lift Apple sales from being sluggish for a long time. Even though there is a lot of exuberance about Apple Intelligence, prudent investors need to remember that the momo crowd focuses only on the bullish factors and ignores the negative factors.&nbsp; In contrast, smart money takes into account both positive and negative factors. There are several negative factors about Apple that the stock market is ignoring right now that prudent investors should be aware of: Apple intelligence will likely not be available in the European Union and China. Even in the U.S., most Apple Intelligence features will not be available right away. The launch of all Apple Intelligence features is staggered over a long period of time. The fact that Apple Intelligence that will initially be present in the iPhone is only half baked may delay the super cycle. The marketing hype about Apple Intelligence is way more than reality. Apple does not own its AI.&nbsp; Apple is licensing its AI from OpenAI. The model behind the AI that Apple is offering is not running on the iPhone or on Apple servers.&nbsp; The model appears to be running on &nbsp;NVIDIA Corp (NASDAQ:NVDA) based servers owned by Microsoft Corp (NASDAQ:MSFT). Apple has been receiving $20B per year from Alphabet Inc Class C (NASDAQ:GOOG) to make Google the default search.&nbsp; A judge has ruled that Google is a monopolist.&nbsp; In due course, the government may force Google to stop paying $20B to Apple. In China, Apple is losing market share to Huawei. Many Chinese believe that Huawei phones are superior to the iPhone. Huawei is trying to crash Apple's iPhone part by introducing the world's first tri-fold phone. Due to geopolitics between the U.S. and China, nationalist sentiment in China is rising against the U.S.&nbsp; This poses a significant risk to Apple since about 20% of Apple sales come from China. We understand that investors are very loyal to Apple.&nbsp; Before sending us hate email for doing a comprehensive analysis, note that Apple is the largest position in The Arora Report&#8217;s ZYX Buy Model Portfolio.&nbsp; Long time members of The Arora Report are long AAPL stock from $4.68. Note that Warren Buffett has been selling AAPL stock. Prudent investors should also be aware that Apple can easily manipulate its stock.&nbsp; After the last Apple event when Apple had carefully staged to run up its stock, Apple stock started falling.&nbsp; Apple aggressively bought its own stock to cause a technical breakout.&nbsp;&nbsp; Apple is clearly anxious about today's event, as demonstrated by the fact that this event is taking place on a Monday, whereas Apple generally does not hold major events on Mondays.&nbsp; Apple is trying to get ahead of a potential European Commission decision on Apple being forced to pay $14B in taxes.&nbsp;&nbsp; There is aggressive buying in stocks this morning due to two reasons: A short squeeze is occurring. The technical pattern with the stock market, especially AI stocks traced on Friday afternoon, historically leads to a selloff on Monday.&nbsp;&nbsp;When stock futures did not open much lower on Sunday night, a short squeeze started.&nbsp; As the night went on, the short squeeze accelerated. The reason stock futures did not open much lower is because Japanese yen weakened.&nbsp; The weakening yen has saved AI stocks from a blood bath this morning.&nbsp;&nbsp; The fact that a short squeeze and weakening yen saved the stock market means that prudent investors should not become complacent.&nbsp; Pay attention to the protection band.&nbsp; See the Protection Band And What To Do Now section below.&nbsp;&nbsp;&nbsp;&nbsp; Magnificent Seven Money Flows In the early trade, money flows are positive in Amazon.com, Inc. (NASDAQ:AMZN), NVDA, MSFT, GOOG, Meta Platforms Inc (NASDAQ:META), and Tesla Inc (NASDAQ:TSLA). In the early trade, money flows are negative in AAPL. In the early trade, money flows are positive in SPDR S&amp;P 500 ETF Trust (NYSE:SPY) and Invesco QQQ Trust Series 1 (NASDAQ:QQQ). Momo Crowd And Smart Money In Stocks Investors can gain an edge by knowing money flows in SPY and QQQ. Investors can get a bigger edge by knowing when smart money is buying stocks, gold, and oil. The most popular ETF for gold is SPDR Gold Trust (NYSE:GLD). The most popular ETF for silver is iShares Silver Trust (NYSE:SLV). The most popular ETF for oil is United States Oil ETF (NYSE:USO). Bitcoin Bitcoin (CRYPTO: BTC) is range bound.&nbsp; Bitcoin ETFs have seen a negative money flow of $1.2B in the last eight days.&nbsp; This is the longest streak of negative money flows in bitcoins ETFs.&nbsp;&nbsp;&nbsp; Protection Band And What To Do Now It is important for investors to look ahead and not in the rearview mirror. Consider continuing to hold good, very long term, existing positions. Based on individual risk preference, consider a protection band consisting of cash or Treasury bills or short-term tactical trades as well as short to medium term hedges and short term hedges. This is a good way to protect yourself and participate in the upside at the same time. You can determine your protection bands by adding cash to hedges.&nbsp; The high band of the protection is appropriate for those who are older or conservative. The low band of the protection is appropriate for those who are younger or aggressive.&nbsp; If you do not hedge, the total cash level should be more than stated above but significantly less than cash plus hedges. A protection band of 0% would be very bullish and would indicate full investment with 0% in cash.&nbsp; A protection band of 100% would be very bearish and would indicate a need for aggressive protection with cash and hedges or aggressive short selling. It is worth reminding that you cannot take advantage of new upcoming opportunities if you are not holding enough cash.&nbsp; When adjusting hedge levels, consider adjusting partial stop quantities for stock positions (non ETF); consider using wider stops on remaining quantities and also allowing more room for high beta stocks.&nbsp; High beta stocks are the ones that move more than the market. Traditional 60/40 Portfolio Probability based risk reward adjusted for inflation does not favor long duration strategic bond allocation at this time. Those who want to stick to traditional 60% allocation to stocks and 40% to bonds may consider focusing on only high quality bonds and bonds of five year duration or less.&nbsp; Those willing to bring sophistication to their investing may consider using bond ETFs as tactical positions and not strategic positions at this time. The Arora Report is known for its accurate calls. The Arora Report correctly called the big artificial intelligence rally before anyone else, the new bull market of 2023, the bear market of 2022, new stock market highs right after the virus low in 2020, the virus drop in 2020, the DJIA rally to 30,000 when it was trading at 16,000, the start of a mega bull market in 2009, and the financial crash of 2008. Please click here to sign up for a free forever Generate Wealth Newsletter.",
   "Alphabet, Inc. (NASDAQ:GOOGL) (NASDAQ:GOOG) last week flagged higher capex spending as it battles to gain supremacy in the artificial intelligence arena, and nervy investors sold off the stock on worries concerning a potential margin contraction. A tech venture capitalist said on Monday that Apple Inc. (NASDAQ:AAPL) could be fairly insulated because of the strategy it has chosen. What Happened: Alphabet, Amazon, Inc. (NASDAQ:AMZN), Microsoft Corp. (NASDAQ:MSFT) and Meta Platforms, Inc. (NASDAQ:META) are on track to increase capex by about 43% this year, said Deepwater Asset Management&#8217;s Gene Munster in a post on X, formerly Twitter. Apple is a standout among the mega-cap techs, he suggested. &#8220;In what seems like it's operating in an alternate universe, $AAPL's CapEx is expected to be down 9% this year as the company monetizes its distribution in exchange for OpenAI's models and infrastructure,&#8221; he said. &#8220;This is one of the reasons why Apple should see record margins sometime in the middle of next year.&#8221; In the March quarter, the company reported gross margin of 46.6%. Talk about bang for your buck: $GOOG $AMZN $MSFT $META will on average increase CapEx this year by 43%. In what seems like it's operating in an alternate universe, $AAPL's CapEx is expected to be down 9% this year as the company monetizes its distribution in exchange for&mdash; Gene Munster (@munster_gene) July 29, 2024 See Also: Best Artificial Intelligence Stocks Why It&#8217;s Important: Apple lagged behind its mega-cap tech peers in embracing AI. On Monday, Cupertino released a new software called Apple Intelligence in the developer beta of iOS 18.1 and the suite of AI features made available will help improve Siri, automatically generate emails and images and sort notifications. The software is expected to be released to the public later this year but it may not be part of the iPhone 16 that will be launched this fall and will be powered by the iOS 18. As Munster said, Apple announced in mid-June that it has forged a partnership with Sam Altman-led OpenAI to integrate ChatGPT into the iPhone, iPad and Mac. The arrangement reportedly does not require Apple to pay for the OpenAI association, with the latter making money through distribution. Apple ended Monday&#8217;s session up 0.13% at $218.24, according to Benzinga Pro data. Read Next: Apple Rolls Out iOS 18.1 Developer Beta With Apple Intelligence, Ahead Of Q3 Earnings Image via Apple",
   'American Airlines Group Inc.(NASDAQ:AAL) shares are trading nearly flat today. The company reportedly implemented a ground stop for all flights early today because of a communication problem. However, American Airlines stated earlier that the outage did not affect its operations, reported Reuters. Related: Microsoft Outage Wreaks Havoc On World As 911, International Airlines, Banks And Airport Services Disrupted (UPDATED) The outage began at approximately 6 p.m. ET on Thursday, affecting a subset of its customers with issues in multiple Azure services in the Central U.S. region. Azure is a cloud computing platform that offers services for building, deploying, and managing applications and services. Other low-cost carriers, including Frontier Group Holdings Inc. (NASDAQ:ULCC), Allegiant Travel Company (NASDAQ:ALGT), and&nbsp;Sun Country Airlines Holdings, Inc. (NASDAQ:SNCY), experienced outages that affected their day-to-day operations. Investors can&nbsp;gain access to&nbsp;the stock via&nbsp;U.S.&nbsp;Global Jets ETF (NYSE:JETS) and&nbsp;First Trust Nasdaq Transportation ETF (NASDAQ:FTXR). Also Read: CrowdStrike Shares Trim Heavy Losses As Microsoft Announces Resolution Of Global Tech Outage: &#8216;This Is Clearly A Major Black Eye&#8217; Price Action: AAL shares are up 0.09% at $10.61 premarket at the last check Friday. Disclaimer: This content was partially produced with the help of AI tools and was reviewed and published by Benzinga editors. Photo via Shutterstock',
   "Shares of Apple, Inc. (NASDAQ:AAPL) fell modestly in premarket trading on Friday, and this assumes importance because all of its Magnificent 7 peers are staring down the barrel. On Thursday, after the market closed, Apple announced that its fiscal year 2024 third-quarter revenue and earnings per share both surpassed expectations and showed growth compared to the previous year. The stock did not respond positively in the after-hours session, as skeptics highlighted concerns about weakness in China. On the earnings call, Team Tim Cook guided September quarter revenue above the Street estimate and also talked up the company&#8217;s AI prospects. Reacting to the results, Wedbush analyst Daniel Ives upped his price target for the stock from $275 to $28 and maintained an Outperform rating. The analyst said it was a &#8220;Gold-medal performance,&#8221; adding that the conference call was upbeat. He sees the stronger September guidance as just the &#8220;opening act for the main event which is an AI-driven super cycle starting with iPhone 16 launching in mid-September.&#8221; Tech venture capitalist and Deepwater Asset Management Managing Partner Gene Munster said, &#8220;Apple's flywheel is flying,&#8221; said Munster, referring to the active installed base that hit a new record. &#8220;Big picture is it's all about FY25 and FY26 iPhone, Mac and iPad powered by Apple Intelligence.&#8221; See Also: Everything You Need to Know About Apple Stock In premarket trading, Apple stock edged down 0.09% to $218.17, according to Benzinga Pro data. Among peers: Nvidia Corp. (NASDAQ:NVDA) fell 4.40% to $104.40. Alphabet Inc. (NASDAQ:GOOGL) (NASDAQ:GOOG) fell 1.94% to $167.45. Microsoft Corporation (NASDAQ:MSFT) slipped 1.88% to $409.25. Amazon.com, Inc. (NASDAQ:AMZN) tumbled 8.36% to $168.68. Meta Platforms, Inc. (NASDAQ:META) slipped a more modest 0.72% to $494.12. Tesla, Inc. (NASDAQ:TSLA) fell 0.82% to $215.09. Read Next: Chip Stocks Staring At Another Bloodbath As Intel Tumbles, Nvidia Dips 4%, ASML Crumbles Over 6%: What&#8217;s Ailing The Sector Friday Photo: Peppinuzzo/Shutterstock",
   'Apple Inc. (NASDAQ:AAPL) is losing its mojo in China&#8217;s crucial market as its domestic players gain ground. The uncertainty regarding the availability of the Apple Intelligence feature in the country is an additional downer. The iPhone maker&#8217;s market share in China plunged to sixth place from third in the second quarter of 2024. According to Canalys, Apple had a market share of 14%, down by two basis points year-on-year. Also Read: China&#8217;s Markets Still Manage To Get Nvidia AI Chips Via Smuggling: Report China&#8217;s smartphone shipment grew 10% year-on-year to over 70 million units in the second quarter. Vivo emerged as the number one with a 19% market share by shipping 13.1 million units, up by 15% year-on-year, driven by solid performance in offline channels and robust online sales during the Chinese e-commerce festival. OPPO remained in second position with a 16% market share after shipping 11.3 million units. HONOR came third with a market share of 15%, backed by shipments of 10.7 million units, implying a 4% growth year-on-year. Huawei Technologies Co bagged the fourth position with a 15% market share supported by shipments of 10.6 million units, which grew by 41% year-on-year. Xiaomi Corp (OTC:XIACF) (OTC:XIACY) came fifth with a market share of 14%, backed by a 17% year-on-year shipment increase to 10 million units. Canalys attributed the local brands&#8217; success to the early significant discounts and promotions backed by offline channel enhancements. The local brands continue to tap the Chinese premium smartphone market by integrating generative artificial intelligence, and their deep collaboration with regional supply chains is a significant advantage, Canalys noted. Canalys also flagged Apple&#8217;s struggle to maintain adequate inventory levels, keep retail prices in check, and maintain the margins of channel partners. Huawei&#8217;s $2,800 Mate XT trifold phone launching on September 20 coincides with the release of the iPhone 16 models. Apple&#8217;s iPhone 16 Pro Max&nbsp;starts at $1,199, while the standard iPhone 16 is $799. International Data Corporation (IDC)&nbsp;expects global smartphone shipments to increase by 5.8% to 1.23 billion units in 2024, marking a recovery with generative AI posing as a significant catalyst. However, due to higher competition in China, Android growth is expected to beat Apple iOS. As per IDC, Apple&#8217;s success hinges on its capability to forge&nbsp;local AI partnerships&nbsp;in China. Ivan Lam, senior analyst at Counterpoint Research told SCMP that iPhone market share in the premium segment will likely decline as Huawei and other Chinese brands gain traction. Apple has not yet revealed its plans for AI integration in China, which could place it at a disadvantage in the race to implement AI, SCMP cites Canalys analyst Lucas Zhong. However, Lam noted that pent-up demand for iPhone upgrades may still boost sales in China. Price Action: AAPL stock closed at $222.66 on Wednesday. image via Shutterstock']]}"""


In [8]:
d = {}
d['val'] = s1['key']
d

{'val': [["To gain an edge, this is what you need to know today. Hope Pinned On Apple Intelligence Please click here for an enlarged chart of Apple Inc (NASDAQ:AAPL). Note the following: This article is about the big picture, not an individual stock.&nbsp; The chart of AAPL stock is being used to illustrate the point. The chart shows when the iPhone 15 was launched. The chart shows the move from iPhone 15 launch to today's iPhone 16 launch. Investors are hopeful that the move in AAPL stock from the iPhone 16 launch to the iPhone 17 will be much bigger compared to the stock move between the launch of iPhone 15 and iPhone 16. The chart shows that even though investors have stars in their eyes, for the short term, AAPL stock has traced a negative pattern. The chart shows that AAPL stock hit the top of the resistance zone in July. The chart shows that AAPL stock failed to penetrate the resistance zone. The chart shows that after AAPL stock failed to penetrate the resistance zone, it fell a

In [3]:
import ast
s1 = ast.literal_eval(s)

In [6]:
s1['key'][0][0]

"To gain an edge, this is what you need to know today. Hope Pinned On Apple Intelligence Please click here for an enlarged chart of Apple Inc (NASDAQ:AAPL). Note the following: This article is about the big picture, not an individual stock.&nbsp; The chart of AAPL stock is being used to illustrate the point. The chart shows when the iPhone 15 was launched. The chart shows the move from iPhone 15 launch to today's iPhone 16 launch. Investors are hopeful that the move in AAPL stock from the iPhone 16 launch to the iPhone 17 will be much bigger compared to the stock move between the launch of iPhone 15 and iPhone 16. The chart shows that even though investors have stars in their eyes, for the short term, AAPL stock has traced a negative pattern. The chart shows that AAPL stock hit the top of the resistance zone in July. The chart shows that AAPL stock failed to penetrate the resistance zone. The chart shows that after AAPL stock failed to penetrate the resistance zone, it fell all the way

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 model and tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

d:\Raghu Studies\FinancialAdvisor\.src_financial_advisor\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
text = " ".join(results['documents'][0])

In [8]:
inputs=tokenizer.encode("sumarize: " +text,return_tensors='pt', max_length=2048, truncation=True)

In [9]:
output = model.generate(inputs, min_length=80, max_length=512)

In [11]:
from pprint import pprint
summary=tokenizer.decode(output[0], skip_special_tokens=True)
pprint(summary)

('a short squeeze is occurring in the stock market, and the weakening yen has '
 'saved the stock. smart money is buying stocks, silver, and oil. a protection '
 'band of 0% would be bullish and would indicate full investment with 0% in '
 'cash. a protection band of 100% would be bearish and would indicate a need '
 'for aggressive short selling.')


In [23]:
type(summary)
import json

In [24]:
summary

'a short squeeze is occurring in the stock market, and the weakening yen has saved the stock. smart money is buying stocks, silver, and oil. a protection band of 0% would be bullish and would indicate full investment with 0% in cash. a protection band of 100% would be bearish and would indicate a need for aggressive short selling.'

In [25]:
from kafka import KafkaProducer
producer = KafkaProducer(
        bootstrap_servers='localhost:9092',
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    
producer.send('quickstart-events', summary)
producer.flush()

In [26]:
from kafka import KafkaConsumer

In [27]:
consumer = KafkaConsumer(
    'quickstart-events',               # Topic name
    bootstrap_servers='localhost:9092',  # Kafka broker
    auto_offset_reset='earliest',        # Start at the earliest available message
    enable_auto_commit=True,             # Automatically commit offsets
    group_id='my-quickstart-group',      # Consumer group ID
    value_deserializer=lambda x: x.decode('utf-8')  # Decode message from bytes to string
    )

In [28]:
for message in consumer:
    # Print consumed message
    print(f"Message consumed: {message.value} of type {type(message)} from partition {message.partition}, offset {message.offset}")
    data = message.value
    break

KeyboardInterrupt: 

In [1]:
import ast
ast.literal_eval('{"text":"mykey","key":"value"}')

{'text': 'mykey', 'key': 'value'}